In [110]:
import pandas as pd
import numpy as np
import sqlite3
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import sqlite3
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
import statsmodels
import statsmodels.api as sm
from statsmodels.multivariate.manova import MANOVA
import math

In [111]:
# Connect to the database. If it does not exist, it will be created
conn = sqlite3.connect(r'C:\Users\15404\Documents\GitHub\research_project\sql_db\test3.db')
# Create a cursor object. This is how we interact with the database
curs = conn.cursor()


### IMPORTANT!!! ###
# By default, sqlite does not enforce foreign key constraints. 
# According to the documentation, this is for backwards compatibility. You have to turn them on yourself.
curs.execute('PRAGMA foreign_keys=ON;')

fulldf = pd.read_sql("""SELECT *
                        FROM tPlantDNA AS t1
                        INNER JOIN tSoilNutrients AS t2 ON t1.sample_id = t2.sample_id
                        INNER JOIN tSample AS t3 ON t1.sample_id = t3.sample_id;""", conn)

conn.close()

In [112]:
fulldf = fulldf.T.drop_duplicates().T

In [113]:
nutrients = fulldf.columns.drop(['ENR', 'V1','V2','PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'sample_id', 'species', 'site_id', 'collect_date']) 
fulldf[nutrients] = fulldf[nutrients].astype(float)

In [114]:
df = fulldf

In [115]:
df[nutrients] = StandardScaler().fit_transform(df[nutrients])  

## By Site Soil Nutrient PCA

In [116]:
X = df[nutrients]

pca = PCA(n_components=2)
components = pca.fit_transform(X)

fig = px.scatter(components, x=0, y=1, color=df['site_id'])
fig.update_layout(
    xaxis_title="PC1 " + str(round(100*pca.explained_variance_ratio_[0], 1)) + '%',
    yaxis_title="PC2 " + str(round(100*pca.explained_variance_ratio_[1], 1)) + '%')
fig.show()

## PCA by Broad Geographic Location

In [117]:
df.shape

(160, 27)

In [118]:
df['site_id'].unique()

array(['RRL', 'FRW', 'MMP', 'PNR', 'CMB', 'SLG', 'RF', 'LFS', 'HR', 'PTW',
       'LM', 'MKP', 'RGT'], dtype=object)

In [119]:
subdf = df
subdf['site_id'].replace({'CMB': 1, 'FRW': 1, 'LFS': 1, 'LM': 0, 'MMP': 1, 'MKP': 2,
                    'PNR': 2, 'PTW': 0, 'RF': 2, 'RGT': 0, 'RRL': 0, 'SGC': 0, 'SLG': 1,'HR': 3}, inplace=True) #
subdf = subdf[subdf["site_id"].isin([0,1,2,3])]

In [120]:
subdf['site_id'].unique()

array([0, 1, 2, 3], dtype=int64)

In [121]:
subdf['site_id'] = subdf['site_id'].replace([0], 'Wintergreen')
subdf['site_id'] = subdf['site_id'].replace([1], 'Cole Mountain')
subdf['site_id'] = subdf['site_id'].replace([2], 'Blacksburg')
subdf['site_id'] = subdf['site_id'].replace([3], 'HR')

In [122]:
subdf['site_id'].unique()

array(['Wintergreen', 'Cole Mountain', 'Blacksburg', 'HR'], dtype=object)

In [123]:
X = subdf[nutrients]

pca = PCA(n_components=2)
components = pca.fit_transform(X)

fig = px.scatter(components, x=0, y=1, color=subdf['site_id'])
fig.update_layout(
    xaxis_title="PC1 " + str(round(100*pca.explained_variance_ratio_[0], 1)) + '%',
    yaxis_title="PC2 " + str(round(100*pca.explained_variance_ratio_[1], 1)) + '%')
fig.show()

## PCA By Species Using Genetic Data

In [124]:
subdf['species_dna'] = subdf['V2']

In [125]:
subdf.head()

,sample_id,V1,V2,species,PC1,PC2,PC3,PC4,PC5,OM,...,Sulfur,Sodium,Zinc,Manganese,Iron,Copper,Boron,site_id,collect_date,species_dna
0,RRL1,0.753244,0.246756,exaltata,0.194769,0.048276,-0.02388,-0.017138,-0.039842,0.053029,...,-1.041732,5.664035,-0.864184,0.070207,-0.443548,3.144685,-0.660275,Wintergreen,6/25/2021,0.246756
1,RRL9,0.581366,0.418634,exaltata,0.097719,0.094733,-0.00656,0.012822,-0.022248,-0.118378,...,-0.292285,0.166878,-0.876883,0.157455,0.123343,2.361486,-0.936831,Wintergreen,6/25/2021,0.418634
2,RRL17,0.591513,0.408487,exaltata,0.099527,0.091996,0.004098,-0.021347,0.00719,-0.394533,...,-1.191622,-0.094891,-0.813387,0.484634,-0.656133,0.571316,-0.936831,Wintergreen,6/25/2021,0.408487
3,FRW3,0.048066,0.951934,syriaca,-0.139465,-0.064022,-0.009963,-0.047536,0.024757,-0.727824,...,-0.891843,1.213956,-0.330815,-0.322408,0.619373,1.242630,0.999055,Cole Mountain,6/25/2021,0.951934
4,FRW12,0.003946,0.996054,syriaca,-0.155135,0.055455,0.009907,-0.043334,-0.009337,-0.661166,...,-1.191622,0.952186,-0.546703,-0.344220,-0.018380,0.571316,0.999055,Cole Mountain,6/25/2021,0.996054


In [132]:
subdf['species_dna'] = np.round(subdf['species_dna'], 2)

TypeError: loop of ufunc does not support argument 0 of type float which has no callable rint method